In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
df_regular = pd.read_csv('NBA_stats.csv',sep=",", encoding='unicode_escape')

In [3]:
df_regular.head()

,Player,Pos,Age,Team,Games,Minutes Played,Fields Goal,Fields Goal Attempted,3-points Field Goal,3-points Field Goal Attempted,...,Defensive Rebounds,Total Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points,Rank,Year
0,Ãlex Abrines,SG,24,OKC,75,15.1,1.5,3.9,1.1,2.9,...,1.2,1.5,0.4,0.5,0.1,0.3,1.7,4.7,1,2018
1,Quincy Acy,PF,27,BRK,70,19.4,1.9,5.2,1.5,4.2,...,3.1,3.7,0.8,0.5,0.4,0.9,2.1,5.9,2,2018
2,Steven Adams,C,24,OKC,76,32.7,5.9,9.4,0.0,0.0,...,4.0,9.0,1.2,1.2,1.0,1.7,2.8,13.9,3,2018
3,Bam Adebayo,C,20,MIA,69,19.8,2.5,4.9,0.0,0.1,...,3.8,5.5,1.5,0.5,0.6,1.0,2.0,6.9,4,2018
4,Arron Afflalo,SG,32,ORL,53,12.9,1.2,3.1,0.5,1.3,...,1.2,1.2,0.6,0.1,0.2,0.4,1.1,3.4,5,2018


In [4]:
unique_team = df_regular['Team'].sort_values().unique()
print(unique_team)

['ATL' 'BOS' 'BRK' 'CHI' 'CHO' 'CLE' 'DAL' 'DEN' 'DET' 'GSW' 'HOU' 'IND'
 'LAC' 'LAL' 'MEM' 'MIA' 'MIL' 'MIN' 'NOP' 'NYK' 'OKC' 'ORL' 'PHI' 'PHO'
 'POR' 'SAC' 'SAS' 'TOR' 'TOT' 'UTA' 'WAS']


TOT means the total of the stats for the relevant season. If a player changes teams in a season, there will be a TOT row in the Team column.

Rk is index for each row.

In [5]:
df_regular.drop(columns=['Rank'], inplace=True)
df_regular = df_regular.loc[df_regular['Team'] != 'TOT']

In [6]:
df_regular.shape

(2444, 24)

In [7]:
df_regular_18 = df_regular[df_regular['Year']==2018]

In [8]:
df_regular_18.shape

(605, 24)

In [9]:
df_regular_19 = df_regular[df_regular['Year']==2019]

In [10]:
df_regular_20 = df_regular[df_regular['Year']==2020]

In [11]:
df_regular_21 = pd.read_csv('2021-2022 NBA Player Stats - Regular.csv',sep=";", encoding="Windows-1252")

In [12]:
df_regular_21.drop(columns=['Rk'], inplace=True)
df_regular_21 = df_regular_21.loc[df_regular_21['Tm'] != 'TOT']

In [13]:
df_regular_22 = pd.read_csv('2022-2023 NBA Player Stats - Regular.csv',sep=";", encoding="Windows-1252")

In [14]:
df_regular_22.drop(columns=['Rk'], inplace=True)
df_regular_22 = df_regular_22.loc[df_regular_22['Tm'] != 'TOT']

These are the 30 teams

Assumption: We will combine 2 years of data, so we have 60 rows in total.
    Each team plays 82 games with the rest 29 teams in the whole regular season. 
    Success of team is defined by total number of wins. This number is manually keyed in from NBA website.
Hypothesis: Teams with more star players, strong defense and efficient offense are more likely to win.
Star player is at player level, and defense and offense is at team level. 
To calculate team level data, we can sum all the goals from all players and then divide by 82 to get team performance.
    Start player is defined as points per game >18 or total rebound per game >10 or assist per game >10
    Defense is defined as total rebounds per game, total blocks per game and total steals per game
    Offense is definded as total assists per game, total 3-point field goals per game
    field goal percentage = total field goal/total field goal attempts from all players
    

Add total rebounds,blocks, steals,assists,3-point field goals,field goals, field goal attempts  of each player as new columns
Total rebound = games played * rebound per game, round up to nearest integer

In [15]:
df_regular_21['TRBD']= round(df_regular_21['TRB']* df_regular_21['G'])
df_regular_21['TBLK']= round(df_regular_21['BLK']* df_regular_21['G'])
df_regular_21['TAST']= round(df_regular_21['AST']* df_regular_21['G'])
df_regular_21['TSTL']= round(df_regular_21['STL']* df_regular_21['G'])
df_regular_21['T3P']= round(df_regular_21['3P']* df_regular_21['G'])
df_regular_21['TFG']= round(df_regular_21['FG']* df_regular_21['G'])
df_regular_21['TFGA']= round(df_regular_21['FGA']* df_regular_21['G'])

In [16]:
df_regular_21.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TOV,PF,PTS,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA
0,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,0.439,...,1.2,2.1,9.1,474.0,44.0,80.0,36.0,58.0,263.0,606.0
1,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,0.547,...,1.5,2.0,6.9,760.0,61.0,258.0,68.0,0.0,213.0,388.0
2,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,0.557,...,2.6,3.1,19.1,566.0,45.0,190.0,78.0,0.0,409.0,728.0
3,Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,0.402,...,0.5,1.1,4.1,86.0,10.0,22.0,6.0,6.0,54.0,131.0
4,LaMarcus Aldridge,C,36,BRK,47,12,22.3,5.4,9.7,0.550,...,0.9,1.7,12.9,258.0,47.0,42.0,14.0,14.0,254.0,456.0


In [17]:
summary_21 = df_regular_21.groupby(['Tm'])[['TRBD','TBLK','TAST','TSTL','T3P']].sum()/82

In [18]:
fgpct_21 = df_regular_21.groupby(['Tm'])[['TFG','TFGA']].sum()
fgpct_21['FG%'] = fgpct_21['TFG']/fgpct_21['TFGA']

In [19]:
summary_21 = summary_21.join(fgpct_21,on = 'Tm',how='left')
summary_21.head()

,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%
Tm,,,,,,,,
ATL,44.109756,4.329268,24.500000,6.975610,13.024390,3403.0,7237.0,0.470222
BOS,46.085366,5.853659,24.865854,7.292683,13.317073,3345.0,7161.0,0.467114
BRK,44.487805,5.414634,25.402439,7.170732,11.487805,3453.0,7248.0,0.476407
CHI,42.402439,4.134146,23.792683,7.121951,10.634146,3427.0,7120.0,0.481320
CHO,44.560976,4.768293,28.195122,8.548780,13.865854,3511.0,7507.0,0.467697


In [20]:
df_regular_22['TRBD']= round(df_regular_22['TRB']* df_regular_22['G'])
df_regular_22['TBLK']= round(df_regular_22['BLK']* df_regular_22['G'])
df_regular_22['TAST']= round(df_regular_22['AST']* df_regular_22['G'])
df_regular_22['TSTL']= round(df_regular_22['STL']* df_regular_22['G'])
df_regular_22['T3P']= round(df_regular_22['3P']* df_regular_22['G'])
df_regular_22['TFG']= round(df_regular_22['FG']* df_regular_22['G'])
df_regular_22['TFGA']= round(df_regular_22['FGA']* df_regular_22['G'])
summary_22 = df_regular_22.groupby(['Tm'])[['TRBD','TBLK','TAST','TSTL','T3P']].sum()/82
fgpct_22 = df_regular_22.groupby(['Tm'])[['TFG','TFGA']].sum()
fgpct_22['FG%'] = fgpct_22['TFG']/fgpct_22['TFGA']
summary_22 = summary_22.join(fgpct_22,on = 'Tm',how='left')
summary_22.head()

,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%
Tm,,,,,,,,
ATL,44.378049,4.865854,25.085366,7.097561,10.829268,3677.0,7570.0,0.485733
BOS,45.463415,5.378049,26.719512,6.487805,16.097561,3456.0,7281.0,0.474660
BRK,40.560976,6.207317,25.573171,7.304878,12.634146,3393.0,6973.0,0.486591
CHI,42.451220,4.560976,24.414634,7.902439,10.304878,3486.0,7119.0,0.489676
CHO,44.536585,5.182927,25.121951,7.743902,10.731707,3369.0,7416.0,0.454288


In [21]:
df_regular_18.rename(columns={"Team": "Tm"}, inplace=True)
df_regular_18.columns

/var/folders/p4/vh8fk7192z31gxtv40_npv4h0000gn/T/ipykernel_4246/4037812582.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regular_18.rename(columns={"Team": "Tm"}, inplace=True)


Index(['Player', 'Pos', 'Age', 'Tm', 'Games', 'Minutes Played', 'Fields Goal',
       'Fields Goal Attempted', '3-points Field Goal',
       '3-points Field Goal Attempted', '2-points Field Goal',
       '2-points Field Goal Attempted', 'Free Throws', 'Free Throws Attempted',
       'Offensive Rebounds', 'Defensive Rebounds', 'Total Rebounds', 'Assists',
       'Steals', 'Blocks', 'Turnovers', 'Personal Fouls', 'Points', 'Year'],
      dtype='object')

In [22]:
df_regular_18['TRBD']= round(df_regular_18['Total Rebounds']* df_regular_18['Games'])
df_regular_18['TBLK']= round(df_regular_18['Blocks']* df_regular_18['Games'])
df_regular_18['TAST']= round(df_regular_18['Assists']* df_regular_18['Games'])
df_regular_18['TSTL']= round(df_regular_18['Steals']* df_regular_18['Games'])
df_regular_18['T3P']= round(df_regular_18['3-points Field Goal']* df_regular_18['Games'])
df_regular_18['TFG']= round(df_regular_18['Fields Goal']* df_regular_18['Games'])
df_regular_18['TFGA']= round(df_regular_18['Fields Goal Attempted']* df_regular_18['Games'])
summary_18 = df_regular_18.groupby(['Tm'])[['TRBD','TBLK','TAST','TSTL','T3P']].sum()/82
fgpct_18 = df_regular_18.groupby(['Tm'])[['TFG','TFGA']].sum()
fgpct_18['FG%'] = fgpct_18['TFG']/fgpct_18['TFGA']
summary_18 = summary_18.join(fgpct_18,on = 'Tm',how='left')
summary_18.head()

/var/folders/p4/vh8fk7192z31gxtv40_npv4h0000gn/T/ipykernel_4246/2417756416.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regular_18['TRBD']= round(df_regular_18['Total Rebounds']* df_regular_18['Games'])
/var/folders/p4/vh8fk7192z31gxtv40_npv4h0000gn/T/ipykernel_4246/2417756416.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regular_18['TBLK']= round(df_regular_18['Blocks']* df_regular_18['Games'])
/var/folders/p4/vh8fk7192z31gxtv40_npv4h0000gn/T/ipykernel_4246/2417756416.py:3: SettingWithCo

,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%
Tm,,,,,,,,
ATL,41.853659,4.256098,23.731707,7.658537,11.158537,3127.0,7018.0,0.445569
BOS,44.500000,4.560976,22.378049,7.329268,11.365854,3149.0,6975.0,0.451470
BRK,44.463415,4.792683,23.756098,6.317073,12.731707,3153.0,7119.0,0.442899
CHI,44.646341,3.439024,23.548780,7.621951,11.097561,3161.0,7286.0,0.433846
CHO,45.451220,4.414634,21.487805,6.853659,10.036585,3201.0,7104.0,0.450591


In [23]:
df_regular_19.rename(columns={"Team": "Tm"}, inplace=True)
df_regular_19['TRBD']= round(df_regular_19['Total Rebounds']* df_regular_19['Games'])
df_regular_19['TBLK']= round(df_regular_19['Blocks']* df_regular_19['Games'])
df_regular_19['TAST']= round(df_regular_19['Assists']* df_regular_19['Games'])
df_regular_19['TSTL']= round(df_regular_19['Steals']* df_regular_19['Games'])
df_regular_19['T3P']= round(df_regular_19['3-points Field Goal']* df_regular_19['Games'])
df_regular_19['TFG']= round(df_regular_19['Fields Goal']* df_regular_19['Games'])
df_regular_19['TFGA']= round(df_regular_19['Fields Goal Attempted']* df_regular_19['Games'])
summary_19 = df_regular_19.groupby(['Tm'])[['TRBD','TBLK','TAST','TSTL','T3P']].sum()/82
fgpct_19 = df_regular_19.groupby(['Tm'])[['TFG','TFGA']].sum()
fgpct_19['FG%'] = fgpct_19['TFG']/fgpct_19['TFGA']
summary_19 = summary_19.join(fgpct_19,on = 'Tm',how='left')
summary_19.head()

/var/folders/p4/vh8fk7192z31gxtv40_npv4h0000gn/T/ipykernel_4246/2817773438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regular_19.rename(columns={"Team": "Tm"}, inplace=True)
/var/folders/p4/vh8fk7192z31gxtv40_npv4h0000gn/T/ipykernel_4246/2817773438.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regular_19['TRBD']= round(df_regular_19['Total Rebounds']* df_regular_19['Games'])
/var/folders/p4/vh8fk7192z31gxtv40_npv4h0000gn/T/ipykernel_4246/2817773438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%
Tm,,,,,,,,
ATL,46.256098,4.963415,25.865854,8.439024,12.975610,3394.0,7530.0,0.450730
BOS,44.414634,5.402439,26.304878,8.695122,12.585366,3456.0,7419.0,0.465831
BRK,46.573171,4.146341,24.036585,6.487805,12.853659,3299.0,7364.0,0.447990
CHI,42.902439,4.292683,21.878049,7.292683,8.963415,3261.0,7201.0,0.452854
CHO,43.817073,4.890244,23.231707,7.146341,11.987805,3300.0,7361.0,0.448309


In [24]:
df_regular_20.rename(columns={"Team": "Tm"}, inplace=True)
df_regular_20['TRBD']= round(df_regular_20['Total Rebounds']* df_regular_20['Games'])
df_regular_20['TBLK']= round(df_regular_20['Blocks']* df_regular_20['Games'])
df_regular_20['TAST']= round(df_regular_20['Assists']* df_regular_20['Games'])
df_regular_20['TSTL']= round(df_regular_20['Steals']* df_regular_20['Games'])
df_regular_20['T3P']= round(df_regular_20['3-points Field Goal']* df_regular_20['Games'])
df_regular_20['TFG']= round(df_regular_20['Fields Goal']* df_regular_20['Games'])
df_regular_20['TFGA']= round(df_regular_20['Fields Goal Attempted']* df_regular_20['Games'])
summary_20 = df_regular_20.groupby(['Tm'])[['TRBD','TBLK','TAST','TSTL','T3P']].sum()/82
fgpct_20 = df_regular_20.groupby(['Tm'])[['TFG','TFGA']].sum()
fgpct_20['FG%'] = fgpct_20['TFG']/fgpct_20['TFGA']
summary_20 = summary_20.join(fgpct_20,on = 'Tm',how='left')
summary_20.head()

/var/folders/p4/vh8fk7192z31gxtv40_npv4h0000gn/T/ipykernel_4246/2250487000.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regular_20.rename(columns={"Team": "Tm"}, inplace=True)
/var/folders/p4/vh8fk7192z31gxtv40_npv4h0000gn/T/ipykernel_4246/2250487000.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regular_20['TRBD']= round(df_regular_20['Total Rebounds']* df_regular_20['Games'])
/var/folders/p4/vh8fk7192z31gxtv40_npv4h0000gn/T/ipykernel_4246/2250487000.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%
Tm,,,,,,,,
ATL,35.329268,4.073171,19.609756,6.500000,9.780488,2721.0,6069.0,0.448344
BOS,40.548780,5.060976,20.170732,7.304878,10.987805,2981.0,6460.0,0.461455
BRK,42.121951,3.804878,21.536585,5.743902,11.487805,2905.0,6502.0,0.446786
CHI,33.158537,3.280488,18.365854,8.000000,9.658537,2578.0,5766.0,0.447104
CHO,34.000000,3.268293,18.890244,5.304878,9.646341,2422.0,5593.0,0.433041


If a player meets any of the following three stats, we encode him as a star player:
points per game >18 or
total rebound per game >10 or
assist per game >10

In [25]:
def star_player(row):
    if row['PTS']>=18:
        return 1
    if row['TRB']>=10:
        return 1
    if row['AST']>=10:
        return 1
    else:
        return 0

In [26]:
def star_players(row):
    if row['Points']>=18:
        return 1
    if row['Total Rebounds']>=10:
        return 1
    if row['Assists']>=10:
        return 1
    else:
        return 0

In [27]:
df_regular_21['Star'] = df_regular_21.apply(star_player, axis=1)
df_regular_21.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,PF,PTS,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,Star
0,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,0.439,...,2.1,9.1,474.0,44.0,80.0,36.0,58.0,263.0,606.0,0
1,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,0.547,...,2.0,6.9,760.0,61.0,258.0,68.0,0.0,213.0,388.0,1
2,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,0.557,...,3.1,19.1,566.0,45.0,190.0,78.0,0.0,409.0,728.0,1
3,Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,0.402,...,1.1,4.1,86.0,10.0,22.0,6.0,6.0,54.0,131.0,0
4,LaMarcus Aldridge,C,36,BRK,47,12,22.3,5.4,9.7,0.550,...,1.7,12.9,258.0,47.0,42.0,14.0,14.0,254.0,456.0,0


In [28]:
star_21 = df_regular_21.groupby(['Tm'])[['Star']].sum()
star_21.head()

,Star
Tm,
ATL,2
BOS,2
BRK,4
CHI,3
CHO,3


In [29]:
summary_21 = summary_21.join(star_21,on = 'Tm',how='left')


In [30]:
df_regular_22['Star'] = df_regular_22.apply(star_player, axis=1)
star_22 = df_regular_22.groupby(['Tm'])[['Star']].sum()
summary_22 = summary_22.join(star_22,on = 'Tm',how='left')
summary_22.head()

,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%,Star
Tm,,,,,,,,,
ATL,44.378049,4.865854,25.085366,7.097561,10.829268,3677.0,7570.0,0.485733,3
BOS,45.463415,5.378049,26.719512,6.487805,16.097561,3456.0,7281.0,0.474660,2
BRK,40.560976,6.207317,25.573171,7.304878,12.634146,3393.0,6973.0,0.486591,3
CHI,42.451220,4.560976,24.414634,7.902439,10.304878,3486.0,7119.0,0.489676,3
CHO,44.536585,5.182927,25.121951,7.743902,10.731707,3369.0,7416.0,0.454288,3


Add total wins each season for each team

In [31]:
df_regular_18['Star'] = df_regular_18.apply(star_players, axis=1)
star_18 = df_regular_18.groupby(['Tm'])[['Star']].sum()
summary_18 = summary_18.join(star_18,on = 'Tm',how='left')
summary_18.head()

/var/folders/p4/vh8fk7192z31gxtv40_npv4h0000gn/T/ipykernel_4246/3632391538.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regular_18['Star'] = df_regular_18.apply(star_players, axis=1)


,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%,Star
Tm,,,,,,,,,
ATL,41.853659,4.256098,23.731707,7.658537,11.158537,3127.0,7018.0,0.445569,1
BOS,44.500000,4.560976,22.378049,7.329268,11.365854,3149.0,6975.0,0.451470,1
BRK,44.463415,4.792683,23.756098,6.317073,12.731707,3153.0,7119.0,0.442899,1
CHI,44.646341,3.439024,23.548780,7.621951,11.097561,3161.0,7286.0,0.433846,0
CHO,45.451220,4.414634,21.487805,6.853659,10.036585,3201.0,7104.0,0.450591,2


In [32]:
df_regular_19['Star'] = df_regular_19.apply(star_players, axis=1)
star_19 = df_regular_19.groupby(['Tm'])[['Star']].sum()
summary_19 = summary_19.join(star_19,on = 'Tm',how='left')
summary_19.head()

/var/folders/p4/vh8fk7192z31gxtv40_npv4h0000gn/T/ipykernel_4246/507605050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regular_19['Star'] = df_regular_19.apply(star_players, axis=1)


,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%,Star
Tm,,,,,,,,,
ATL,46.256098,4.963415,25.865854,8.439024,12.975610,3394.0,7530.0,0.450730,2
BOS,44.414634,5.402439,26.304878,8.695122,12.585366,3456.0,7419.0,0.465831,1
BRK,46.573171,4.146341,24.036585,6.487805,12.853659,3299.0,7364.0,0.447990,1
CHI,42.902439,4.292683,21.878049,7.292683,8.963415,3261.0,7201.0,0.452854,3
CHO,43.817073,4.890244,23.231707,7.146341,11.987805,3300.0,7361.0,0.448309,1


In [33]:
df_regular_20['Star'] = df_regular_20.apply(star_players, axis=1)
star_20 = df_regular_20.groupby(['Tm'])[['Star']].sum()
summary_20 = summary_20.join(star_20,on = 'Tm',how='left')
summary_20.head()

/var/folders/p4/vh8fk7192z31gxtv40_npv4h0000gn/T/ipykernel_4246/731685023.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regular_20['Star'] = df_regular_20.apply(star_players, axis=1)


,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%,Star
Tm,,,,,,,,,
ATL,35.329268,4.073171,19.609756,6.500000,9.780488,2721.0,6069.0,0.448344,2
BOS,40.548780,5.060976,20.170732,7.304878,10.987805,2981.0,6460.0,0.461455,3
BRK,42.121951,3.804878,21.536585,5.743902,11.487805,2905.0,6502.0,0.446786,4
CHI,33.158537,3.280488,18.365854,8.000000,9.658537,2578.0,5766.0,0.447104,1
CHO,34.000000,3.268293,18.890244,5.304878,9.646341,2422.0,5593.0,0.433041,2


In [34]:
wins = pd.read_csv('NBA Standings 18-22.csv',sep=",", encoding='unicode_escape')

In [35]:
wins.head()

,Tm,Wins_18,Wins_19,Wins_20,Wins_21,Wins_22
0,ATL,29,20,41,43,41
1,BOS,49,48,36,51,57
2,BRK,42,35,48,44,45
3,CHO,39,23,33,43,27
4,CHI,22,22,31,46,51


In [36]:
summary_21= summary_21.reset_index()

In [37]:
summary_21 = summary_21.merge(wins[['Tm','Wins_21']],on = 'Tm',how='left')
summary_21.rename(columns={"Wins_21": "Win"}, inplace=True)

In [38]:
summary_22= summary_22.reset_index()
summary_22 = summary_22.merge(wins[['Tm','Wins_22']],on = 'Tm',how='left')
summary_22.rename(columns={"Wins_22": "Win"}, inplace=True)

In [39]:
summary_18= summary_18.reset_index()
summary_18 = summary_18.merge(wins[['Tm','Wins_18']],on = 'Tm',how='left')
summary_18.rename(columns={"Wins_18": "Win"}, inplace=True)

In [40]:
summary_19= summary_19.reset_index()
summary_19 = summary_19.merge(wins[['Tm','Wins_19']],on = 'Tm',how='left')
summary_19.rename(columns={"Wins_19": "Win"}, inplace=True)

In [41]:
summary_20= summary_20.reset_index()
summary_20 = summary_20.merge(wins[['Tm','Wins_20']],on = 'Tm',how='left')
summary_20.rename(columns={"Wins_20": "Win"}, inplace=True)

In [42]:
df= pd.concat([summary_18,summary_19,summary_20,summary_21, summary_22])
df.shape

(150, 11)

In [43]:
df.head()

,Tm,TRBD,TBLK,TAST,TSTL,T3P,TFG,TFGA,FG%,Star,Win
0,ATL,41.853659,4.256098,23.731707,7.658537,11.158537,3127.0,7018.0,0.445569,1,29
1,BOS,44.500000,4.560976,22.378049,7.329268,11.365854,3149.0,6975.0,0.451470,1,49
2,BRK,44.463415,4.792683,23.756098,6.317073,12.731707,3153.0,7119.0,0.442899,1,42
3,CHI,44.646341,3.439024,23.548780,7.621951,11.097561,3161.0,7286.0,0.433846,0,22
4,CHO,45.451220,4.414634,21.487805,6.853659,10.036585,3201.0,7104.0,0.450591,2,39


Correlation analysis

In [44]:
df[['Win','TRBD','TBLK','TAST','TSTL','T3P','FG%','Star']].corr()

,Win,TRBD,TBLK,TAST,TSTL,T3P,FG%,Star
Win,1.000000,0.248196,0.223701,0.275600,0.210274,0.233482,0.449938,0.098735
TRBD,0.248196,1.000000,0.438945,0.577776,0.306898,0.344585,0.129919,-0.003722
TBLK,0.223701,0.438945,1.000000,0.396088,0.357175,0.106661,0.267615,0.098584
TAST,0.275600,0.577776,0.396088,1.000000,0.349233,0.375898,0.576404,0.015349
TSTL,0.210274,0.306898,0.357175,0.349233,1.000000,0.018993,0.081256,-0.003838
T3P,0.233482,0.344585,0.106661,0.375898,0.018993,1.000000,0.147174,0.105016
FG%,0.449938,0.129919,0.267615,0.576404,0.081256,0.147174,1.000000,0.221052
Star,0.098735,-0.003722,0.098584,0.015349,-0.003838,0.105016,0.221052,1.000000


All features are positively correlated with target variable.

Field goal percentage is the most correlated feature with # wins.

Total Assist and Total Rebound are highly correlated.

Total Assist and Field goal percentage are highly correlated.


Linear regression

In [45]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [46]:
y = df['Win']
x = df[['TRBD','TBLK','TAST','TSTL','T3P','FG%','Star']]
model = sm.OLS(y,x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    Win   R-squared (uncentered):                   0.933
Model:                            OLS   Adj. R-squared (uncentered):              0.929
Method:                 Least Squares   F-statistic:                              282.8
Date:                Tue, 12 Dec 2023   Prob (F-statistic):                    1.99e-80
Time:                        21:42:21   Log-Likelihood:                         -565.57
No. Observations:                 150   AIC:                                      1145.
Df Residuals:                     143   BIC:                                      1166.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TRBD          -0.1606      0.335     -0.479      0.633      -0.824       0.503
TBLK           1.5019      1.353      1.110      0.269      -1.172       4.176
TAST           0.2974      0.503      0.591      0.555      -0.697       1.292
TSTL           0.9492      1.126      0.843      0.401      -1.277       3.175
T3P            0.8961      0.583      1.538      0.126      -0.256       2.048
FG%           28.9807     29.214      0.992      0.323     -28.767      86.728
Star           0.4877      0.938      0.520      0.604      -1.366       2.342
==============================================================================
Omnibus:                        6.353   Durbin-Watson:                   1.890
Prob(Omnibus):                  0.042   Jarque-Bera (JB):                5.595
Skew:                          -0.395   Prob(JB):                       0.0610
Kurtosis:                       2.479   Cond. No.                     1.71e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.71e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""